In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
import malaya

model = malaya.translation.en_ms.transformer(model = 'small-bigbird')

Downloading:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

2022-10-02 23:11:42.990007: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 23:11:42.993867: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-02 23:11:42.993885: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-10-02 23:11:42.993889: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-10-02 23:11:42.993960: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-10-02 23:11:42.993973: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [5]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [6]:
from tqdm import tqdm

batch_size = 2

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [01:50<00:00,  4.50it/s]


In [7]:
import re

def bigbird_postprocessing(string):
    return re.sub(r'([!,.?/;\'])\1+', r'\1 ', string).strip()

filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(bigbird_postprocessing(r))
        filtered_right.append(right[no])

In [8]:
refs = [filtered_right]
sys = filtered_left

In [9]:
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 36.90195033318057,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '67.0/43.8/30.1/21.0 (BP = 1.000 ratio = 1.028 hyp_len = 22637 ref_len = 22027)',
 'bp': 1.0,
 'counts': [15162, 9473, 6212, 4129],
 'totals': [22637, 21640, 20643, 19646],
 'sys_len': 22637,
 'ref_len': 22027,
 'precisions': [66.9788399522905,
  43.775415896487985,
  30.09252531124352,
  21.017000916217043],
 'prec_str': '67.0/43.8/30.1/21.0',
 'ratio': 1.0276932855132337}

In [10]:
chrf.corpus_score(sys, refs)

chrF2++ = 62.85